In [1]:
import numpy as np
import copy
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit import BasicAer
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_state_qsphere
from scipy.optimize import minimize
import seaborn
import time
from TotalSolver import *
from TotalSolver import GridSearcher
from Testing.InstanceGenerator.InstanceGenerator import create_instance
from ClassicalSolver import dynamic_programming, lazy_greedy
from datetime import datetime
import os
import pandas as pd


Error in callback <function flush_figures at 0x7f8ff980f0d0> (for post_execute):


KeyboardInterrupt: 

In [2]:
TYPES = ["uncor", "weak_cor", "strong_cor", "inv_str", "alm_str", "subset"]
N  = 10
R = 1000
np.random.seed(123)

In [3]:
def run_testing(N):
    data = []
    check_file = open("Testing/Check.txt", "r")
    while True:
        print(N)
        N = max((N + 5) % 40, 10)
        problem_type = TYPES[np.random.randint(0, len(TYPES))]
        profits, weights, max_weight_list = create_instance(problem_type, N, R, H=4)
        items = [Item(i, profits[i], weights[i]) for i in range(len(profits))]
        solving_type = "Copula"
        init_type = "smoothened"
        for mw in max_weight_list[2:]:
            timestamp = str(datetime.fromtimestamp(time.time())).replace(" ", "-")[:19]
            
            qt1 = time.time()
            quantum_solver = GridSearcher(items, mw, [-0.5 for _ in profits], 10, 10, 1, "Copula", "nonbiased")
            quantum_solution = quantum_solver.get_max_result()[0]
            quantum_time = time.time() - qt1
            ax = quantum_solver.visualize()
            print(quantum_time)
            ct1 = time.time()
            classical_sol_opt = dynamic_programming(items, mw)
            classical_opt_time = time.time() - ct1
            ct2 = time.time()
            classical_sol_greedy_string = lazy_greedy(profits, weights, mw)[0]
            classical_sol_greedy = (np.array(classical_sol_greedy_string) * np.array(profits)).sum()
            classical_greedy_time = time.time() - ct2
            dt = time.time()
            data_line = [str(profits), str(weights), str(mw), problem_type, solving_type, init_type, quantum_solution, classical_sol_greedy, classical_sol_opt, quantum_time, classical_greedy_time, classical_opt_time, timestamp]
            data.append(data_line)
            check_file.seek(0)
            df = pd.DataFrame(data, columns=["Profits", "Weights", "Max_Weight", "Problem type", "Solving type", "Init type", "Quantum Solution", "Classical Greedy Solution", "Classical Optimal Solution", "Quantum Runtime", "Classical Greedy Runtime", "Classical Optimal Runtime", "Timestamp"])
            if check_file.read() == "FINISH":
                df.to_excel("Testing/OutcomesGreedy/{}.xlsx".format(timestamp))
                return



In [4]:
run_testing(N) 

10


TypeError: tuple indices must be integers or slices, not tuple